* 출처 : https://keras.io/ko/applications/


In [ ]:
# %%bash
# [ ! -f flower_photos_300x200_small_train_test2.zip ]&& wget https://raw.githubusercontent.com/Finfra/AI_Vision/master/data/flower_photos_300x200_small_train_test2.zip

# rm -rf __MACOSX
# rm -rf flowers
# unzip -q flower_photos_300x200_small_train_test2.zip
# mv flower_photos_300x200_small_train_test2 flowers

# cd flowers


# find .|grep .DS_Store|xargs rm -f
# find .|head -n 10


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/202111_ObjectDection_Kitri/Day2_SourceCode/DogAndCat

test  train


In [3]:
from os import listdir
from os.path import isfile, join, splitext
import cv2
def getFolder(thePath,isFile=True):
    return [f for f in listdir(thePath) if isFile == isfile(join(thePath, f)) ]

def convert(thePath,to_w,to_h):
    ext=splitext(".jpg")[0]
    if ext in ('.jpg','.png'):
      img = cv2.imread(thePath)
      if (to_h,to_w,3) != img.shape :
        img = cv2.resize(img,(to_w,to_h))
        print(img.shape[0],img.shape[1],'to',to_w,to_h , thePath , )
        cv2.imwrite(thePath,img)
      # else:
      #   print(thePath,"is not changed (same)")

def convertAll(tPath,to_w,to_h):
  for folder in getFolder(tPath,False):
    print('-------------------')
    print(join(tPath,folder))
    convertAll(join(tPath,folder),to_w,to_h)
  for files in getFolder(tPath,True):
      convert(join(tPath,files),to_w,to_h)


tPath='/content/drive/MyDrive/202111_ObjectDection_Kitri/Day2_SourceCode/DogAndCat'


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

w=224
h=224
color=3

convertAll(tPath,w,h)



# load and iterate training dataset
datagen = ImageDataGenerator()
train_data = datagen.flow_from_directory(directory='/content/drive/MyDrive/202111_ObjectDection_Kitri/Day2_SourceCode/DogAndCat/train/', class_mode='categorical', batch_size=64,target_size=(h, w))
# load and iterate test dataset
test_data = datagen.flow_from_directory(directory='/content/drive/MyDrive/202111_ObjectDection_Kitri/Day2_SourceCode/DogAndCat/test/',  class_mode='categorical', batch_size=64,target_size=(h,w))



-------------------
/content/drive/MyDrive/202111_ObjectDection_Kitri/Day2_SourceCode/DogAndCat/train
-------------------
/content/drive/MyDrive/202111_ObjectDection_Kitri/Day2_SourceCode/DogAndCat/train/cat
-------------------
/content/drive/MyDrive/202111_ObjectDection_Kitri/Day2_SourceCode/DogAndCat/train/dog
-------------------
/content/drive/MyDrive/202111_ObjectDection_Kitri/Day2_SourceCode/DogAndCat/test
-------------------
/content/drive/MyDrive/202111_ObjectDection_Kitri/Day2_SourceCode/DogAndCat/test/cat
-------------------
/content/drive/MyDrive/202111_ObjectDection_Kitri/Day2_SourceCode/DogAndCat/test/dog
Found 16 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [6]:
for image_batch, label_batch in train_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  label_set_count=label_batch.shape[1]
  break


Image batch shape:  (16, 224, 224, 3)
Label batch shape:  (16, 2)


In [7]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# 선행학습된 기준모델을 만듭니다
base_model = InceptionV3(weights='imagenet', include_top=False)

# 글로벌 공간 평균값 풀링 레이어를 더합니다
x = base_model.output
x = GlobalAveragePooling2D()(x)
# 완전 연결 레이어를 더합니다
x = Dense(1024, activation='relu')(x)
# 로지스틱 레이어를 더합니다 -- 200가지 클래스가 있다고 가정합니다
predictions = Dense(label_set_count, activation='softmax')(x)

# 다음은 학습할 모델입니다
model = Model(inputs=base_model.input, outputs=predictions)

# 첫째로: (난수로 초기값이 설정된) 가장 상위 레이어들만 학습시킵니다
# 다시 말해서 모든 InceptionV3 콘볼루션 레이어를 고정합니다
for layer in base_model.layers:
    layer.trainable = False

# 모델을 컴파일합니다 (*꼭* 레이어를 학습불가 상태로 세팅하고난 *후*에 컴파일합니다)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])



# 모델을 새로운 데이터에 대해 몇 세대간 학습합니다
model.fit_generator(train_data, 
                    validation_data=test_data, 
                    validation_steps=8,
                    epochs=50
)

87924736/87910968 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/50
1/1 [==============================] - 14s 14s/step - loss: 25.5352 - accuracy: 0.5000 - val_loss: 623.8452 - val_accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 93ms/step - loss: 662.5739 - accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 92ms/step - loss: 148.6446 - accuracy: 0.5000
Epoch 4/50
1/1 [==============================] - 0s 93ms/step - loss: 46.9118 - accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 0s 94ms/step - loss: 63.4815 - accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 0s 89ms/step - loss: 24.7806 - accuracy: 0.5000
Epoch 7/50
1/1 [==============================] - 0s 90ms/step - loss: 40.5879 - accuracy: 0.5000
Epoch 8/50
1/1 [==============================] - 0s 100ms/step - loss: 13.3070 - accuracy: 0.6250
Epoch 9/50
1/1 [==============================] - 0s 89ms/step - loss: 13.7716 - accuracy: 0.6250
Epoch 10/50
1/1 [==============================] - 0s 90ms/step - loss:

In [8]:
score = model.evaluate_generator(test_data)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 13.611994743347168
Test accuracy: 0.6000000238418579


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [9]:
# 이 시점에서 상위 레이어들은 충분히 학습이 되었기에,
# inception V3의 콘볼루션 레이어에 대한 파인튜닝을 시작합니다 
# 가장 밑 N개의 레이어를 고정하고 나머지 상위 레이어를 학습시킵니다

# 레이어 이름과 레이어 인덱스를 시각화하여
# 얼마나 많은 레이어를 고정시켜야 하는지 확인합니다:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)



0 input_1
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_14
5

In [10]:
# 가장 상위 2개의 inception 블록을 학습하기로 고릅니다,
# 다시 말하면 첫 249개의 레이어는 고정시키고 나머지는 고정하지 않습니다:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True



In [11]:
# 이러한 수정사항이 효과를 내려면 모델을 다시 컴파일해야 합니다
# 낮은 학습 속도로 세팅된 SGD를 사용합니다
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])

# 다시 한 번 모델을 학습시킵니다
# (이번엔 상위 2개의 inception 블록을 상위의 밀집 레이어들과 함께 파인튜닝합니다)
model.fit_generator(train_data, 
                    validation_data=test_data, 
                    validation_steps=8,
                    epochs=100
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 0.7498 - accuracy: 0.5625 - val_loss: 2.4999 - val_accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 98ms/step - loss: 0.7426 - accuracy: 0.5625
Epoch 3/100
1/1 [==============================] - 0s 95ms/step - loss: 0.7292 - accuracy: 0.5625
Epoch 4/100
1/1 [==============================] - 0s 102ms/step - loss: 0.7105 - accuracy: 0.5625
Epoch 5/100
1/1 [==============================] - 0s 101ms/step - loss: 0.6876 - accuracy: 0.5625
Epoch 6/100
1/1 [==============================] - 0s 102ms/step - loss: 0.6616 - accuracy: 0.6250
Epoch 7/100
1/1 [==============================] - 0s 106ms/step - loss: 0.6336 - accuracy: 0.6250
Epoch 8/100
1/1 [==============================] - 0s 98ms/step - loss: 0.6042 - accuracy: 0.6250
Epoch 9/100
1/1 [==============================] - 0s 101ms/step - loss: 0.5744 - accuracy: 0.6875
Epoch 10/100
1/1 [==============================] - 0s 95ms/step - loss: 

In [12]:
score = model.evaluate_generator(test_data)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.665062427520752
Test accuracy: 0.6000000238418579


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.
